# PCA predictions using logistic regression
to show that the added genetic data does not add much to the PCA we need to apply regression on the PCA values to the phenotype and show that the explenability is not too good

perform analysis on phenotypes produced with:
* 0%/10%/30%/50%/70% weight PCA
* 0%/10%/30%/50%/70% heretability

In [1]:
import pandas as pd
import os
import glob
import re
import re
import statsmodels.api as sm
import datetime


In [2]:
GENETICS_PATH = "/sci/nosnap/michall/roeizucker/covar_tests/small_genetics"
COVARS_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK"
PLINK_PATH = "/sci/nosnap/michall/roeizucker/plink2"
TEST_COVAR = "covariates_sex_year_of_birth_AC_batch_40_PC.txt"
COVAR_DF_PREFIX = "temp_covar_df"
VERSION = "multi_effect_score_0.7"

# RELATIVE_EFFECT_OF_PC =  0.7347749803596619
# TODO: try to add load json for parameters

In [3]:
covar_df = pd.read_csv(os.path.join(COVARS_PATH,TEST_COVAR),sep="\t",header=None)

In [ ]:
import pandas as pd
import glob
import re

# Define the file pattern to match your CSV files.
file_pattern = "temp_covar_df_v=multi_effect_score_*_her=*_cov=*.csv"
file_list = glob.glob(file_pattern)

# Compile a regular expression to extract the numbers from the filename.
pattern = re.compile(
    r"temp_covar_df_v=multi_effect_score_([0-9\.]+)_her=([0-9\.]+)_cov=([0-9\.]+)\.csv"
)

# Dictionary to store DataFrames keyed by (multi_effect_score, her, cov)
dfs = {}

# Loop over each file in the list
for filename in file_list:
    match = pattern.search(filename)
    if match:
        # Extract the numeric values and convert them to floats.
        multi_effect_score = float(match.group(1))
        her = float(match.group(2))
        cov = float(match.group(3))
    else:
        print(f"Filename {filename} did not match the expected pattern.")
        continue  # Skip files that do not match the pattern

    # Read the CSV file into a DataFrame.
    df = pd.read_csv(filename)

    # (Optional) Add columns with the extracted parameter values.
    df["multi_effect_score"] = multi_effect_score
    df["her"] = her
    df["cov"] = cov

    # Use a tuple of parameters as the dictionary key.
    key = (multi_effect_score, her, cov)
    dfs[key] = df

# Example usage: Access the DataFrame for a specific set of parameter values.
# example_key = (0.1, 0.1, 0.1)
# if example_key in dfs:
#     print(f"DataFrame for parameters {example_key}:")
#     print(dfs[example_key].head())
# else:
#     print(f"No DataFrame found for parameters {example_key}")


In [ ]:
regression_results = {}

# Loop over each DataFrame in the dictionary.
for key, df in dfs.items():
    multi_effect_score, her, cov = key
    # Check if her + cov is less than 1.
    if her + cov <= 1:
        # print(key)
        # Define predictor column names: PC1, PC2, ..., PC40.
        pc_columns = [f"PC{i}" for i in range(1, 41)]
        # Extract predictors (X) and response (Y).
        temp_df = df.dropna()
        X = temp_df[pc_columns]
        y = temp_df["disease_indicator"]

        # Add an intercept (constant) to the model.
        X = sm.add_constant(X)

        # Fit the linear regression model.
        model = sm.OLS(y, X).fit()

        # Store the fitted model in the results dictionary.
        regression_results[key] = model

# Example: Present the regression summary for each file that met the condition.


In [ ]:
# for params, model in regression_results.items():
#     print(f"\nRegression results for parameters (multi_effect_score, her, cov) = {params}:\n")
#     print(model.summary())
#     # break


In [ ]:
results = []
columns = ["ver","her","cov","r_sq",'const'] + ["PC" + str(i) for i in range(1,41)] 
for params, model in regression_results.items():
    # if params[0] == 0.2 and params[1] == 0.7:
    # print(f"For parameters (ver, her, cov) = {params}:")
    # print(f"R-squared: {model.rsquared:.3f}")
    # print(f"expected effect: {(RELATIVE_EFFECT_OF_PC * params[2])**2:.6f}")
    # print(model.params)
    ver = params[0]
    her = params[1]
    cov = params[2]
    r_sq = model.rsquared
    coefs = list(model.params)
    
    values = [ver,her,cov,r_sq] + coefs
    results.append(values)
    # print(dir(model))

In [ ]:
df = pd.DataFrame(results,columns=columns)

x = str(datetime.datetime.now())
df.to_csv(f"regression_results_{VERSION}_{x}.csv")

In [ ]:
df

In [ ]:
versions = [0.5]
COVARIATE_SCORES = [0,0.1,0.2,0.3,0.5,0.7]
for ver in versions:
    for cov in COVARIATE_SCORES:
        print("ver",ver,"cov",cov,"mean_rseq",df[(df["ver"] == ver) & (df["cov"] == cov)]["r_sq"].mean())

In [ ]:
conf_matrix = df.groupby(['ver', 'cov'])['r_sq'].mean().unstack('cov')
print(conf_matrix)